Pre Processing function

In [80]:
remove_words = ["@handle", "rt", "http"]

def pre_process(sentence):
    sentence = sentence.split()
    target_remove = set()
    for token in sentence:
        for target in remove_words:
            if target in token:
                target_remove.add(token)
                
    for target in target_remove:
        sentence.remove(target)
    
    sentence = ' '.join(sentence)
    return sentence


Load Train data to train_dict. 
train_dict[id] = [[train_instace1], [train_instance2] ....]

In [81]:
import csv
import collections
import time
import matplotlib.pyplot as plt
time_start=time.time()

train_file_path = "data/train_tweets.txt"
train_dict = collections.defaultdict(list)

empty_sentence_counter = 0
with open(train_file_path, encoding='utf-8') as tsvfile:
    reader = csv.reader((x.replace('\0', '') for x in tsvfile), delimiter='\t')
    for i, row in enumerate(reader):
        id = int(row[0])
        instance = pre_process(row[1].lower())
        if not instance == "":
            train_dict[id].append(instance)
        else:
            #print("Empty sentence row:{0}".format(i+1))
            #print("id: {0}".format(id))
            #print("sentenc: {0}".format(instance))
            empty_sentence_counter += 1
    print("Total rows: %d" % i)
print("Total ids: %d" % len(train_dict))

time_end=time.time()
print("Empty sentence counter: {0}".format(empty_sentence_counter))
print("Time spent: {0:.2f}ms".format((time_end-time_start)*1000))

Total rows: 328194
Total ids: 9292
Empty sentence counter: 998
Time spent: 2074.16ms


In [82]:
print(train_dict[1319])

["i'm looking for a recommendation for a leasing call center other than level one. any suggestions?", "i'm at triple rock brewing in berkeley...great spot to watch the yankees win!", 'whostalkin.com is a great source as well #aptchat', 'i also use google on all our comps #aptchat', 'hi! matt here from recp. i also use google usually receive craigslist ads or ratings from the #aptchat', 'shopping on oakland today....so far i like what i see', 'rt real-time social media stats from @handle', 'does your social class determine your online social network? - #cnn', 'the new paramount theatre sign is up! they are going to light it on 10/21/09 at 6:00 pm!! you should all come!', 'cl now asking for phone numbers to verify your posting.looks like it is localized to socal so far. anyone else seeing this?', 'i humbly remind u that bad reviews r not the problem, but a symptom', "udr's augmented reality search application passes 25,000 downloads", 'off to dempression...any tubers in the house?', 'it 

Split Dev Train
!! There are Some ID have 0 train instance

In [83]:
import random
import numpy as np

dev_split = 0.1
train_split = 1 - dev_split

dev_set_dict = {}
train_set_dict = {}

for id in train_dict:
    target_list = train_dict[id]
    length = len(target_list)
    random.shuffle(target_list)
    split = int(np.ceil(length*dev_split))
    dev_set_dict[id] = target_list[:split]
    train_set_dict[id] = target_list[split:length]
#     print(len(dev_set_dict[id]), len(train_set_dict[id]), length)

print(len(dev_set_dict), len(train_set_dict), len(train_dict))
    
    


9292 9292 9292


In [84]:
def save_to_file(target_dict, file_path):
    file_lines = []
    for id in target_dict:
        for sentence in target_dict[id]:
            file_lines.append(str(id) + '\t' + sentence + '\n')
            
    random.shuffle(file_lines)
    with open(file_path, 'w+') as file:
        for item in file_lines:
            file.write(item)
    return

dev_set_path = 'data/v1/dev_set_v1.txt'
train_set_path = 'data/v1/train_set_v1.txt'

save_to_file(dev_set_dict, dev_set_path)
save_to_file(train_set_dict, train_set_path)
